# 🏥 고양이 의료 문서 분석

이 노트북은 `data/은내과.pdf` 파일을 PaddleOCR로 분석하여 의료 데이터를 추출합니다.

## 주요 기능
- PDF → 이미지 변환
- PaddleOCR를 통한 테이블 데이터 추출
- 의료 검사 결과 구조화
- JSON 형태로 데이터 저장

## 1. 환경 설정 및 라이브러리 로드

In [8]:
# 필요한 라이브러리 설치 (첫 실행 시만)
!pip install paddleocr paddlepaddle pillow pandas

## 2. 파일 로드

In [ ]:
import io
from typing import List, Dict, Any

class UploadedFileManager:
    def __init__(self):
        # 업로드된 파일들이 저장되는 리스트
        # self.files 의 구조 예시:
        # [
        #     {
        #         "name": "blood_test1.png",       # 파일 이름 (str)
        #         "content": b"...",               # 원본 파일 내용 (bytes)
        #         "io": <_io.BytesIO object>       # BytesIO 객체 (PIL/OpenCV 사용 가능)
        #     },
        #     {
        #         "name": "blood_test2.jpg",
        #         "content": b"...",
        #         "io": <_io.BytesIO object>
        #     }
        # ]
        self.files: List[Dict[str, Any]] = []

    def handler(self, change) -> None:
        """
        ipywidgets FileUpload 위젯의 observe 이벤트 핸들러.

        Args:
            change (dict): ipywidgets 가 observe 이벤트에서 전달하는 dict
                change 의 구조 예시:
                {
                    'name': 'value',                # 변경된 속성 이름
                    'old': (),                      # 이전 값 (tuple of UploadedFile)
                    'new': (                        # 새로운 값 (tuple of UploadedFile)
                        UploadedFile(
                            name='blood_test1.png',
                            type='image/png',
                            size=12345,
                            content=b'...'
                        ),
                        UploadedFile(
                            name='blood_test2.jpg',
                            type='image/jpeg',
                            size=23456,
                            content=b'...'
                        )
                    ),
                    'owner': FileUpload(...)        # 이벤트를 발생시킨 위젯 객체
                }

        Returns:
            None: 반환값은 없으며, self.files 에 업로드된 파일 리스트를 저장합니다.
        """
        try:
            if not change or "new" not in change or not change["new"]:
                self.files = []
                return

            files = list(change["new"])  # tuple of UploadedFile → list 변환

            self.files = [
                {
                    "name": f.name,
                    "content": f.content,
                    "io": io.BytesIO(f.content)
                }
                for f in files
            ]

            print("✅ 업로드 완료:", [f["name"] for f in self.files])

        except Exception as e:
            print(f"❌ 파일 처리 오류: {e}")
            self.files = []

In [34]:
from typing import List, Tuple, Any

def format_ocr_result(ocr_result: Any) -> List[Tuple[str, float, List[List[float]]]]:
    """
    PaddleOCR.predict() 결과를 사람이 보기 좋은 (text, confidence, bbox) 리스트로 변환

    Args:
        ocr_result: PaddleOCR.predict() 반환값

    Returns:
        List[Tuple[str, float, List[List[float]]]]:
            - text (str): 인식된 문자열
            - confidence (float): 신뢰도 (0~1)
            - bbox (List[List[float]]): 텍스트 영역 좌표 (사각형 꼭짓점 4개)
    """
    formatted = []

    try:
        for block in ocr_result:
            # block 구조 예시:
            # {
            #   'bbox': [[x1,y1],[x2,y2],[x3,y3],[x4,y4]],
            #   'text': 'Hemoglobin',
            #   'score': 0.9876
            # }
            text = block.get("text", "")
            score = float(block.get("score", 0.0))
            bbox = block.get("bbox", [])
            formatted.append((text, score, bbox))

    except Exception as e:
        print(f"❌ OCR 결과 변환 오류: {e}")

    return formatted

In [35]:
import ipywidgets as widgets
from IPython.display import display

# 1. FileUpload 위젯 생성
uploader = widgets.FileUpload(accept='.png,.jpg,.jpeg', multiple=True)
display(uploader)

# 2. 파일 매니저 인스턴스 생성
file_manager = UploadedFileManager()

# 3. observe 연결
uploader.observe(file_manager.handler, names="value")

FileUpload(value=(), accept='.png,.jpg,.jpeg', description='Upload', multiple=True)

In [51]:

# 4. 업로드 완료 후 파일 목록 접근
import json
print(json.dumps(file_manager.files, ensure_ascii=False, indent=2, default=str))


# 5. PaddleOCR에 바로 넘기기
import numpy as np, cv2
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_textline_orientation=True, lang='ch')

# 업로드된 모든 파일 순회
for f in file_manager.files:
    print(f"📂 Processing: {f['name']}")

    # 이미지 디코딩
    img_array = np.frombuffer(f["content"], np.uint8)
    cv_img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    # 흑백 변환
    gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)

    # 단순 이진화
    # _, thresh = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)

    # 또는 적응형 이진화가 더 잘 먹힐 때가 많습니다
    # thresh = cv2.adaptiveThreshold(gray, 255,
    #                                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    #                                cv2.THRESH_BINARY, 25, 15)

    # 이진화 (가장자리 강조)
    _, binary = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)

    # -------------------------------
    # 1. 수평 라인 제거
    # -------------------------------
    h_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
    detected_lines_h = cv2.morphologyEx(binary, cv2.MORPH_OPEN, h_kernel, iterations=2)
    line_removed_h = cv2.subtract(binary, detected_lines_h)

    # -------------------------------
    # 2. 수직 라인 제거
    # -------------------------------
    v_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
    detected_lines_v = cv2.morphologyEx(line_removed_h, cv2.MORPH_OPEN, v_kernel, iterations=2)
    line_removed = cv2.subtract(line_removed_h, detected_lines_v)

    # -------------------------------
    # 3. 원래 색상으로 복원
    # -------------------------------
    processed = cv2.bitwise_not(line_removed)

    # 확인용 저장
    cv2.imwrite("debug_output/01_gray.png", gray)                # 흑백 이미지
    cv2.imwrite("debug_output/02_binary.png", binary)            # 이진화 결과
    cv2.imwrite("debug_output/03_no_hlines.png", line_removed_h) # 수평선 제거 후
    cv2.imwrite("debug_output/04_no_vlines.png", line_removed)   # 수직선 제거 후
    cv2.imwrite("debug_output/05_processed.png", processed)      # 최종 OCR 입력 이미지

    # OCR 입력용 BGR 변환
    ocr_input = cv2.cvtColor(processed, cv2.COLOR_GRAY2BGR)

    # OCR 실행
    ocr_result = ocr.ocr(ocr_input)

    # 결과 출력
    print("🔎 OCR Result:")
    print(ocr_result)
    print("=" * 50)  # 구분선

    # 사람이 읽기 좋은 형식으로 변환
    readable = format_ocr_result(ocr_result)

    # 결과 출력
    print("🔎 OCR Result:")
    for text, conf, bbox in readable:
        print(f"  • {text} (conf={conf:.2f}) {bbox}")
    print("=" * 50)  # 구분선

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.


[
  {
    "name": "은내과 3.png",
    "content": "<memory at 0x78cc22fb0280>",
    "io": "<_io.BytesIO object at 0x78cc280f9fd0>"
  }
]


Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('PP-OCRv5_server_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-OCRv5_server_rec`.


📂 Processing: 은내과 3.png
